# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,12.32,62,39,5.72,SH,1717126972
1,1,port-aux-francais,-49.3500,70.2167,2.72,82,99,10.10,TF,1717126972
2,2,wajir,1.7471,40.0573,23.98,78,0,1.54,KE,1717126973
3,3,afaahiti,-17.7500,-149.2833,27.81,77,18,3.58,PF,1717126973
4,4,geraldton,-28.7667,114.6000,22.23,40,0,7.20,AU,1717126973


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    frame_width=1000,
    frame_height=600
)

# Display the map
# YOUR CODE HERE
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] < 20) &
    (city_data_df["Max Temp"] > 12) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
# YOUR CODE HERE
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
# YOUR CODE HERE
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,terenos,-20.4422,-54.8603,19.84,52,0,4.12,BR,1717126974
27,27,douglas,41.3000,-96.1670,18.30,92,0,2.57,US,1717126976
58,58,maardu,59.4767,25.0250,18.00,72,0,0.51,EE,1717126980
79,79,east london,-33.0153,27.9116,15.52,63,0,4.12,ZA,1717126983
95,95,nouadhibou,20.9310,-17.0347,18.99,88,0,4.12,MR,1717126985


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ""

# Display sample data
# YOUR CODE HERE
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
10,terenos,BR,-20.4422,-54.8603,52,
27,douglas,US,41.3000,-96.1670,92,
58,maardu,EE,59.4767,25.0250,72,
79,east london,ZA,-33.0153,27.9116,63,
95,nouadhibou,MR,20.9310,-17.0347,88,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel", 
    "limit": 1,                         
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add filter and bias parameters to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    data = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = data["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
terenos - nearest hotel: No hotel found
douglas - nearest hotel: Residence Inn Omaha West
maardu - nearest hotel: Hostel Fortuna
east london - nearest hotel: No hotel found
nouadhibou - nearest hotel: Hotel valencia
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
murfreesboro - nearest hotel: Ramada
bethel - nearest hotel: Hampton Inn Danbury
fort bragg - nearest hotel: Airborne Inn Lodging
hamilton - nearest hotel: North Vista Manor
belyy yar - nearest hotel: No hotel found
hwange - nearest hotel: No hotel found
vadso - nearest hotel: Scandic Vadsø
correntina - nearest hotel: Pousada Silva
saynshand - nearest hotel: No hotel found
komatipoort - nearest hotel: Ecolux Boutique Hotel
mutuali - nearest hotel: No hotel found
ribas do rio pardo - nearest hotel: Hotel São Francisco
qurayyat - nearest hotel: دانة الشمال
tucumcari - nearest hotel: Pow Wow Inn
arapgir - nearest hotel: Arapgir Nazar Hotel
dakhla - nearest hotel: Hotel El Massira
maxixe - neare

,City,Country,Lat,Lng,Humidity,Hotel Name
10,terenos,BR,-20.4422,-54.8603,52,No hotel found
27,douglas,US,41.3000,-96.1670,92,Residence Inn Omaha West
58,maardu,EE,59.4767,25.0250,72,Hostel Fortuna
79,east london,ZA,-33.0153,27.9116,63,No hotel found
95,nouadhibou,MR,20.9310,-17.0347,88,Hotel valencia
99,fortuna,US,40.5982,-124.1573,79,Comfort Inn & Suites Redwood Country
141,murfreesboro,US,35.8456,-86.3903,74,Ramada
146,bethel,US,41.3712,-73.4140,77,Hampton Inn Danbury
162,fort bragg,US,35.1390,-79.0060,75,Airborne Inn Lodging
181,hamilton,US,39.1834,-84.5333,78,North Vista Manor


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
# Filter to remove rows where "Hotel Name" is "No hotel found"
filtered_hotel_df = hotel_df[hotel_df["Hotel Name"] != "No hotel found"]

# Create "Hotel Info" column
filtered_hotel_df["Hotel Info"] = list(
    zip(filtered_hotel_df["Lat"], filtered_hotel_df["Lng"], filtered_hotel_df["Hotel Name"], filtered_hotel_df["City"])
)

hotel_map = filtered_hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True, 
    tiles="OSM",
    hover_cols=["Hotel Name", "City", "Country"],
    size=100,
    alpha=0.7,
    frame_width=1000,
    frame_height=600
)

# Display the map
# YOUR CODE HERE
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,City,Country)